In [ ]:
import pandas as pd

In [4]:
df = pd.read_csv('../data/raw/dados.csv')

In [15]:
df.head()

,sql_registration_number,transaction_nature,transaction_value_declared,transaction_date,reference_market_value,transmitted_proportion,proportional_reference_market_value,adopted_calculation_base,financing_type,financed_value,...,address_name,address,state,district,zone,lat,lng,city,city_ibge,ddd
0,13944000322,1.Compra e venda,264000.0,2022-12-22 00:00:00,15828870.0,0.20,31657.7400,264000.0,1.Sistema Financeiro de Habitação,166320.00,...,Nordestina,Avenida Nordestina,SP,Vila Nova Curuçá,leste,-23.51802,-46.42164,São Paulo,3550308.0,11.0
1,6726500084,1.Compra e venda,800000.0,2022-12-23 00:00:00,846885.0,100.00,846885.0000,846885.0,1.Sistema Financeiro de Habitação,400000.00,...,Juripiranga,Rua Juripiranga,SP,Parque Vitória,norte,-23.47005,-46.59494,São Paulo,3550308.0,11.0
2,13944000322,1.Compra e venda,203700.0,2022-12-22 00:00:00,15828870.0,0.01,1582.8870,203700.0,1.Sistema Financeiro de Habitação,144059.51,...,Nordestina,Avenida Nordestina,SP,Vila Nova Curuçá,leste,-23.51802,-46.42164,São Paulo,3550308.0,11.0
3,5614900926,1.Compra e venda,264000.0,2022-12-22 00:00:00,3403192.0,0.08,2722.5536,264000.0,1.Sistema Financeiro de Habitação,183400.00,...,Antônio de Barros,Rua Antônio de Barros,SP,Vila Carrão,leste,-23.53885,-46.55771,São Paulo,3550308.0,11.0
4,13944000322,1.Compra e venda,200550.0,2022-12-22 00:00:00,15828870.0,0.01,1582.8870,200550.0,1.Sistema Financeiro de Habitação,100433.00,...,Nordestina,Avenida Nordestina,SP,Vila Nova Curuçá,leste,-23.51802,-46.42164,São Paulo,3550308.0,11.0


In [ ]:
df.tail()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175457 entries, 0 to 175456
Data columns (total 35 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   sql_registration_number              175457 non-null  int64  
 1   transaction_nature                   175457 non-null  object 
 2   transaction_value_declared           175457 non-null  float64
 3   transaction_date                     175457 non-null  object 
 4   reference_market_value               175457 non-null  float64
 5   transmitted_proportion               175457 non-null  float64
 6   proportional_reference_market_value  175457 non-null  float64
 7   adopted_calculation_base             175457 non-null  float64
 8   financing_type                       72551 non-null   object 
 9   financed_value                       175457 non-null  float64
 10  registry_office                      175457 non-null  object 
 11  property_regi

In [9]:
df.shape

(175457, 35)

In [5]:
df.columns

Index(['sql_registration_number', 'transaction_nature',
       'transaction_value_declared', 'transaction_date',
       'reference_market_value', 'transmitted_proportion',
       'proportional_reference_market_value', 'adopted_calculation_base',
       'financing_type', 'financed_value', 'registry_office',
       'property_registration', 'sql_status', 'land_area_m2', 'frontage_m',
       'ideal_fraction', 'built_area_m2', 'iptu_use', 'iptu_use_description',
       'iptu_standard', 'iptu_standard_description', 'iptu_acc', 'cep',
       'number', 'address_type', 'address_name', 'address', 'state',
       'district', 'zone', 'lat', 'lng', 'city', 'city_ibge', 'ddd'],
      dtype='object')

In [10]:
df.isnull().sum()

sql_registration_number                     0
transaction_nature                          0
transaction_value_declared                  0
transaction_date                            0
reference_market_value                      0
transmitted_proportion                      0
proportional_reference_market_value         0
adopted_calculation_base                    0
financing_type                         102906
financed_value                              0
registry_office                             0
property_registration                       0
sql_status                                  0
land_area_m2                                0
frontage_m                                  0
ideal_fraction                              0
built_area_m2                               0
iptu_use                                    0
iptu_use_description                        0
iptu_standard                               0
iptu_standard_description                   0
iptu_acc                          

In [13]:
df.value_counts()

sql_registration_number  transaction_nature  transaction_value_declared  transaction_date     reference_market_value  transmitted_proportion  proportional_reference_market_value  adopted_calculation_base  financing_type                     financed_value  registry_office                     property_registration  sql_status     land_area_m2  frontage_m  ideal_fraction  built_area_m2  iptu_use  iptu_use_description                                 iptu_standard  iptu_standard_description                                iptu_acc  cep      number  address_type  address_name                          address                                       state  district         zone    lat        lng        city       city_ibge  ddd 
30105400571              1.Compra e venda    264000.0                    2022-11-18 00:00:00  11830043.0              0.01                    1183.0043                            264000.0                  2.Minha Casa Minha Vida            211200.0        11º Cartório de 

In [14]:
df.describe(include='object')

,transaction_nature,transaction_date,financing_type,registry_office,sql_status,iptu_use_description,iptu_standard_description,address_type,address_name,address,state,district,zone,city
count,175457,175457,72551,175457,175457,175457,175457,175436,175436,175436,175436,175436,175436,175436
unique,32,1179,3,26,5,34,11,15,17793,17851,1,1447,5,1
top,1.Compra e venda,2023-06-30 00:00:00,1.Sistema Financeiro de Habitação,11º Cartório de Registro de Imóvel,Ativo Predial,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),RESIDENCIAL VERTICAL,Rua,André de Almeida,Rua André de Almeida,SP,Vila Andrade,sul,São Paulo
freq,157449,1983,59576,23921,143244,62026,73379,130414,1567,1567,175436,4390,57570,175436


Preprocessamento e Limpeza

___

Análise

___

Modelagem

___

Análise Prescritiva

___